In [1]:
# Pacotes de manipulação de dados
import numpy as np
import pandas as pd

In [10]:
# Carregar os dados da aba específica do arquivo Excel
df = pd.read_excel('/home/rodrigo/Downloads/DRE_PowerBI.xlsx', sheet_name='Fato.RazaoContabil')

# Remover as colunas
colunas_para_remover = ['Lote', 'Doc.', 'SEQ', 'Histórico']
df.drop(columns=colunas_para_remover, inplace=True)

# Converter a coluna 'data' para Timestamp
try:
    df['Data'] = pd.to_datetime(df['data'])
except ValueError as e:
    print(f"Não foi possível converter a coluna 'data' para Timestamp: {e}")

# Renomear as colunas
df.rename(columns={
    'CR': 'Centro_Resultado',
    'Descrição CR': 'Descricao_Centro_Resultado',
    'Descrião Conta': 'Descricao_Conta',
    'Débito': 'Debito',
    'Crédito': 'Credito'
}, inplace=True)

# Exibir o número de linhas e colunas
print('Número de linhas e colunas:')
print(df.shape)

print('\n-------------------------------------')

# Exibir os tipos de dados ajustados
print(df.dtypes)

print('\n-------------------------------------')

# Exibir as primeiras linhas dos dados
print('\nVisualização inicial dos dados:')
print(df.head())

Número de linhas e colunas:
(58813, 8)

-------------------------------------
Centro_Resultado                       int64
Descricao_Centro_Resultado            object
Conta                                 object
Descrição Conta                       object
Data                          datetime64[ns]
Debito                               float64
Credito                              float64
Saldo                                float64
dtype: object

-------------------------------------

Visualização inicial dos dados:
   Centro_Resultado Descricao_Centro_Resultado     Conta  \
0               211       Futebol Profissional  3.1.1.01   
1               211       Futebol Profissional  3.1.1.01   
2               211       Futebol Profissional  3.1.1.01   
3               211       Futebol Profissional  3.1.1.01   
4               211       Futebol Profissional  3.1.1.01   

                  Descrição Conta       Data  Debito  Credito      Saldo  
0  Campeonato Baiano - Bilheteria 2022-0

In [9]:
# Amostra de dados da base
df.sample(5, random_state=42)

,Centro_Resultado,Descricao_Centro_Resultado,Conta,Descrição Conta,Data,Debito,Credito,Saldo
31588,301,Loja Esquadrão,3.9.4.01,ICMS,2022-06-08,17.98,NaN,1253104.67
17104,301,Loja Esquadrão,3.9.3.02,Descontos concedidos - Loja Esquadrão,2022-04-19,69.97,NaN,1085058.22
26602,301,Loja Esquadrão,3.9.4.01,ICMS,2022-04-13,25.92,NaN,1080267.03
50168,211,Futebol Profissional,4.1.1.10,Direito de Imagem,2022-05-31,56000.00,NaN,53621779.64
29449,301,Loja Esquadrão,3.9.4.01,ICMS,2022-05-20,56.68,NaN,1176011.09
